In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.tree import DecisionTreeClassifier

In [39]:
df = pd.read_csv('./data/final_earthquake_volcano.csv')

In [40]:
# for modelling we did not need: year, lat and lon, country and closest volcanic names
df=df.drop(['eq_year','ve_year',
          'eq_lat','eq_lon','ve_lat','ve_lon',
          've_country','ve_closest'], axis=1)

In [41]:
df.head()

,eq_depth,eq_mag,ve_region,ve_type,ve_ele,ve_rock_chem,ve_tect_zone,ve_tect_crust,eq_ve_dist,eq_ve_yeardiff,potential_eruption
0,131.6,6.0,"Japan, Taiwan, Marianas",Submarine,-8,Intermediate,Subduction Zone,Intermediate Crust,112.162847,24,0
1,80.0,5.8,Indonesia,Stratovolcano,1325,Intermediate,Subduction Zone,Oceanic Crust,51.898694,51,0
2,20.0,6.2,New Zealand to Fiji,Caldera,515,Intermediate,Subduction Zone,Oceanic Crust,147.078303,49,1
3,15.0,5.8,Antarctica,Shield,1370,Mafic,Subduction Zone,Oceanic Crust,176.936371,42,0
4,15.0,5.8,Philippines and SE Asia,Stratovolcano(es),1565,Intermediate,Subduction Zone,Continental Crust,274.261200,51,0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30230 entries, 0 to 30229
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   eq_depth            30230 non-null  float64
 1   eq_mag              30230 non-null  float64
 2   ve_region           30230 non-null  object 
 3   ve_type             30230 non-null  object 
 4   ve_ele              30230 non-null  int64  
 5   ve_rock_chem        30230 non-null  object 
 6   ve_tect_zone        30230 non-null  object 
 7   ve_tect_crust       30230 non-null  object 
 8   eq_ve_dist          30230 non-null  float64
 9   eq_ve_yeardiff      30230 non-null  int64  
 10  potential_eruption  30230 non-null  int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 2.5+ MB


In [45]:
categorical = ['ve_region','ve_type','ve_rock_chem','ve_tect_zone','ve_tect_crust']
for i in categorical:
    df[i] = df[i].astype('category')
df.dtypes

eq_depth               float64
eq_mag                 float64
ve_region             category
ve_type               category
ve_ele                   int64
ve_rock_chem          category
ve_tect_zone          category
ve_tect_crust         category
eq_ve_dist             float64
eq_ve_yeardiff           int64
potential_eruption       int64
dtype: object

In [47]:
for i in categorical:
    df[i] = df[i].cat.codes
df.head()

,eq_depth,eq_mag,ve_region,ve_type,ve_ele,ve_rock_chem,ve_tect_zone,ve_tect_crust,eq_ve_dist,eq_ve_yeardiff,potential_eruption
0,131.6,6.0,8,18,-8,1,2,1,112.162847,24,0
1,80.0,5.8,7,14,1325,1,2,2,51.898694,51,0
2,20.0,6.2,15,0,515,1,2,2,147.078303,49,1
3,15.0,5.8,2,12,1370,2,2,2,176.936371,42,0
4,15.0,5.8,16,15,1565,1,2,0,274.261200,51,0


In [48]:
# for modelling we did not need: year, lat and lon, country and closest volcanic names
y=df["potential_eruption"]
X=df.drop(["potential_eruption"], axis=1)


In [49]:
y.value_counts() # imbalanced value

0    27217
1     3013
Name: potential_eruption, dtype: int64

In [50]:
# train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, stratify=y, random_state=20, test_size=0.25)

### Baseline - LogisticRegression

In [99]:
logreg=LogisticRegression(max_iter = 3000)
logreg.fit(X_train, y_train)
logreg_pred=logreg.predict(X_test)
print(classification_report(y_test, logreg_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      6805
           1       0.80      0.62      0.70       753

    accuracy                           0.95      7558
   macro avg       0.88      0.80      0.84      7558
weighted avg       0.94      0.95      0.94      7558



In [129]:
logreg_tuned=LogisticRegression(max_iter = 3000, fit_intercept = True, solver = 'lbfgs')

In [130]:
logreg_tuned.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

In [131]:
logreg_tuned_pred=logreg.predict(X_test)

In [132]:
print(classification_report(y_test, logreg_tuned_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      6805
           1       0.80      0.62      0.70       753

    accuracy                           0.95      7558
   macro avg       0.88      0.80      0.84      7558
weighted avg       0.94      0.95      0.94      7558



We are predicting wether an earthquake could trigger volcanic eruption. it would be better to stay cautious for natural disaster prediction, therefore our aim is higher recall score on value 1.

### Tuning - LogisticRegression

In [101]:
gridsearch= GridSearchCV(estimator=logreg,
                        param_grid={
                            "solver" : ['newton-cg', 'lbfgs', 'sag', 'saga'], 
                            "fit_intercept":[True, False]
                        },
                         scoring="accuracy",
                         cv=5,
                         n_jobs=-1
                        )

gridsearch.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=3000), n_jobs=-1,
             param_grid={'fit_intercept': [True, False],
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
             scoring='accuracy')

In [102]:
# It seems the default settings turned out to be the best parameter on LogisticRegression
gridsearch.best_params_

{'fit_intercept': True, 'solver': 'lbfgs'}

In [103]:
gridsearch.best_score_

0.944028044387402

### SMOTE - LogisticRegression
Next step is to try dealing with our imbalanced data with SMOTE

In [106]:
# we should try to deal with imbalanced data with SMOTE
smote=SMOTE()
X_smote, y_smote=smote.fit_sample(X_train,y_train)

In [107]:
logreg=LogisticRegression(max_iter = 3000)
logreg.fit(X_smote, y_smote)
logreg_pred=logreg.predict(X_test)
print(classification_report(y_test, logreg_pred))

              precision    recall  f1-score   support

           0       0.99      0.92      0.96      6805
           1       0.57      0.92      0.71       753

    accuracy                           0.92      7558
   macro avg       0.78      0.92      0.83      7558
weighted avg       0.95      0.92      0.93      7558



In [122]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

ClassReport = []

for train_index , test_index in kf.split(X_train,y_train):
    X_train_kf,X_test_kf,y_train_kf,y_test_kf = X_train.iloc[train_index],X_train.iloc[test_index],y_train.iloc[train_index],y_train.iloc[test_index]
    model = LogisticRegression(max_iter = 3000)
    model.fit(X_train_kf,y_train_kf)
    pred = logreg.predict(X_test_kf)

    ClassReport.append(classification_report(y_test_kf,pred))

    
n = 1
for item in ClassReport:
  print('================ Fold Number %d ================' %(n))
  n += 1
  print(item)
  print()

================ Fold Number 1 ================
              precision    recall  f1-score   support

           0       0.99      0.92      0.96      4058
           1       0.59      0.92      0.72       477

    accuracy                           0.92      4535
   macro avg       0.79      0.92      0.84      4535
weighted avg       0.95      0.92      0.93      4535


================ Fold Number 2 ================
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      4111
           1       0.58      0.92      0.71       424

    accuracy                           0.93      4535
   macro avg       0.78      0.92      0.83      4535
weighted avg       0.95      0.93      0.94      4535


================ Fold Number 3 ================
              precision    recall  f1-score   support

           0       0.99      0.94      0.96      4066
           1       0.63      0.93      0.75       468

    accuracy                        

C:\Users\adinatha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [108]:
import pickle


In [110]:
filename = 'final_model'
pickle.dump(logreg, open(filename, 'wb'))

In [116]:
filename = 'x_dummies_column.sav'
pickle.dump(X_smote.columns,open(filename, 'wb'))

In [115]:
filename = 'real_column.sav'
pickle.dump(df.drop('potential_eruption',axis = 1).columns, open(filename, 'wb'))

In [134]:
import joblib

In [135]:
joblib.dump(logreg,'model_joblib')

['model_joblib']

In [133]:
X.head()

,eq_depth,eq_mag,ve_region,ve_type,ve_ele,ve_rock_chem,ve_tect_zone,ve_tect_crust,eq_ve_dist,eq_ve_yeardiff
0,131.6,6.0,8,18,-8,1,2,1,112.162847,24
1,80.0,5.8,7,14,1325,1,2,2,51.898694,51
2,20.0,6.2,15,0,515,1,2,2,147.078303,49
3,15.0,5.8,2,12,1370,2,2,2,176.936371,42
4,15.0,5.8,16,15,1565,1,2,0,274.261200,51
